In [ ]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure


class DatabaseConnection:
    def __init__(self,username, password, host='localhost', port='27017'):
        # Credenciales del docker-compose
        username = username
        password = password
        host = host
        port = port
        
        # URI completa con autenticación
        self.uri = f'mongodb://{username}:{password}@{host}:{port}/'
        self.client = None
        self.db = None
    
    def connect(self, db_name):
        try:
            self.client = MongoClient(
                self.uri,
                serverSelectionTimeoutMS=5000,
                authSource='admin'  # Importante: autenticarse contra la DB 'admin'
            )
            
            # Verificar conexión
            self.client.admin.command('ping')
            self.db = self.client[db_name]
            print(f"✓ Conectado a la base de datos '{db_name}'")
            return self.db
            
        except ConnectionFailure as e:
            print(f"No se pudo conectar a MongoDB: {e}")
            return None
        except OperationFailure as e:
            print(f"Error de autenticación: {e}")
            return None
    
    def close(self):
        if self.client:
            self.client.close()
            print("Conexión cerrada")

In [6]:
db_conn = DatabaseConnection()
db = db_conn.connect('universidad')

try:
    resultado = db.estudiantes.insert_one({"nombre": "Juan", "edad": 25})
    print(f"✓ Documento insertado: {resultado.inserted_id}")
except Exception as e:
    print(f"❌ ERROR al insertar: {e}")
    # Probablemente verás: "command insert requires authentication"


✓ Conectado a la base de datos 'universidad'
✓ Documento insertado: 6984c032d0178b3e06025f02
